In [10]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score

In [11]:
data = pd.read_excel("data.xlsx")

In [12]:
X=data.iloc[:,:-1]
y=data.iloc[:,-1]

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [14]:
# Create a function to build the model
def build_model(hidden_units=100):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(hidden_units, activation='relu', input_shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(1)  # Output layer with one neuron for regression
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [36]:
# Define the hyperparameters for grid search
param_grid = {
    'epochs': np.arange(25, 101, 25),           # 50,100 ... 500
    'batch_size': [2**i for i in range(3, 7)]  # Powers of 2: 16, 32, 64, 128, 256
}

In [37]:
# Create the KerasRegressor model
keras_reg = tf.keras.wrappers.scikit_learn.KerasRegressor(build_fn=build_model)

/tmp/ipykernel_38881/2303772581.py:2: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  keras_reg = tf.keras.wrappers.scikit_learn.KerasRegressor(build_fn=build_model)


In [38]:
# Use GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(estimator=keras_reg, param_grid=param_grid, scoring='neg_mean_squared_error', cv=3, verbose=1)

In [ ]:
# Fit the model to the data
grid_result = grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
Epoch 1/25
7/7 [==============================] - 1s 4ms/step - loss: 6032.2085
Epoch 2/25
7/7 [==============================] - 0s 5ms/step - loss: 4162.9087
Epoch 3/25
7/7 [==============================] - 0s 4ms/step - loss: 2809.1565
Epoch 4/25
7/7 [==============================] - 0s 4ms/step - loss: 1845.7800
Epoch 5/25
7/7 [==============================] - 0s 4ms/step - loss: 1203.2743
Epoch 6/25
7/7 [==============================] - 0s 4ms/step - loss: 906.0663
Epoch 7/25
7/7 [==============================] - 0s 4ms/step - loss: 703.6362
Epoch 8/25
7/7 [==============================] - 0s 4ms/step - loss: 608.2356
Epoch 9/25
7/7 [==============================] - 0s 4ms/step - loss: 519.3821
Epoch 10/25
7/7 [==============================] - 0s 4ms/step - loss: 449.7851
Epoch 11/25
7/7 [==============================] - 0s 4ms/step - loss: 388.8586
Epoch 12/25
7/7 [==============================] - 0s 4ms/step 

In [32]:
# Get the best parameters from the grid search
best_params = grid_result.best_params_
print(f'Best Parameters: {best_params}')

Best Parameters: {'batch_size': 8, 'epochs': 75}


In [33]:
# Make predictions on the testing set using the best model
y_test_pred = grid_result.predict(X_test)
y_train_pred=grid_result.predict(X_train)

10/10 [==============================] - 0s 3ms/step


In [34]:
# Calculate Mean Squared Error and R-squared
mse_train=mean_squared_error(y_train,y_train_pred)
r2_train=r2_score(y_train,y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)
r2_test = r2_score(y_test, y_test_pred)

In [35]:
print(f'Training MSE: {mse_train:.2f}, R-squared: {r2_train:.2f}')
print(f'Testing MSE: {mse_test:.2f}, R-squared: {r2_test:.2f}')

Training MSE: 30.18, R-squared: 0.89
Testing MSE: 15.86, R-squared: 0.95
